In [ ]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
base = Path('/mnt/data/repo-basis')
gc = pd.read_csv(base/'data/gc_rate.csv', parse_dates=['date'])
fut = pd.read_csv(base/'data/future_bund.csv', parse_dates=['date','expiry'])
bond = pd.read_csv(base/'data/bond_ctd.csv', parse_dates=['date','coupon_next_date'])
repo_isin = pd.read_csv(base/'data/repo_isin.csv', parse_dates=['date'])
df = gc.merge(fut,on='date').merge(bond,on='date').merge(repo_isin,on='date')
df['days_to_expiry'] = (df['expiry']-df['date']).dt.days.clip(lower=1)
from importlib.machinery import SourceFileLoader
repo = SourceFileLoader('repo', str(base/'src/repo.py')).load_module()
signals = SourceFileLoader('signals', str(base/'src/signals.py')).load_module()
irr = []
for _,row in df.iterrows():
    irr.append(repo.implied_repo_rate(row.F,row.CF,row.clean,row.AI,row.AI,int(row.days_to_expiry)))
df['IRR'] = irr
df['signal'] = [signals.signal_irr_vs_gc(i,g) for i,g in zip(df['IRR'],df['gc_on'])]
df[['date','F','clean','AI','gc_on','IRR','signal']].head()